In [ ]:
import pandas as pd
from pyprojroot import here

import matplotlib.pyplot as plt
from utils import 

In [ ]:
data = pd.read_csv(here() / 'data/base/data.csv')

In [ ]:
data = data.iloc[:,68:73].replace({'Unchecked':0, 'Checked':1})

In [ ]:
data.columns = data.columns.str[55:-1]

In [ ]:
series = ((data.sum() / data.sum().sum()) * 100)

In [ ]:
fig, ax = plt.subplots(figsize=(4,3))

series.plot.barh(
    ax=ax, 
    color='lightgrey',
    width=.6)

ax.set_xlim([0,100])

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.savefig(here() / 'output/plots/interventions.png', dpi=300, bbox_inches='tight')